In [1]:
### IMPORTS ###

Num GPUs Available:  1


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import scipy # This is new!
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
import scipy.integrate as integrate
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model,Sequential,Model
from tensorflow.keras.utils import load_img, img_to_array 
import os
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications import MobileNetV2,VGG16
from keras.applications import InceptionV3
from datetime import datetime

In [ ]:
### IMPORTS ###

In [ ]:
### DATASET GENERAL FUNCTION ###

In [ ]:
def datasetGeneralFunction(trainPath,validatePath,bSize,sizeY,sizeX):
    
    trainData_path = trainPath
    validateData_path = validatePath

    trainDatagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 10,
        width_shift_range =  0.15,
        height_shift_range =  0.15,
        shear_range = 0.5,
        zoom_range = 0.1,
        channel_shift_range = 15,
        horizontal_flip = True
    )
    testDatagen = ImageDataGenerator(rescale=1./255)


    trainGenerator = trainDatagen.flow_from_directory(
        trainData_path,
        target_size = (sizeY,sizeX),
        batch_size = bSize,
        class_mode = 'binary'
    )
    validationGenerator = testDatagen.flow_from_directory(
        validateData_path,
        target_size = (sizeY,sizeX),
        batch_size = bSize,
        class_mode = 'binary'
    )
    
    return trainGenerator, validationGenerator

In [ ]:
### DATASET GENERAL FUNCTION ###

In [ ]:
### ARCHITECTURES ###

In [ ]:
def modelKarol(inputY,inputX,channel):    
    model = Sequential()

    # First Block
    model.add(Conv2D(32, kernel_size=(3,3), strides=(2,2), input_shape=(inputY,inputX,channel), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2D(32, kernel_size=(3,3), strides=(2,2), input_shape=(300,300,3), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(2,2), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

  
    model.add(Conv2D(64, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())


    model.add(Conv2D(64, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(2,2), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2D(128, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(2,2), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())


    model.add(Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(2,2), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, kernel_size=(1,1), strides=(1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def modelMobile(inputY,inputX,channel):

    base_model = MobileNetV2(input_shape=(inputY, inputX, channel), include_top=False, weights='imagenet')

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def modelVGG(inputY,inputX,channel):
    input_shape = (inputY, inputX, channel)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
def modelGoogleNet(inputY,inputX,channel):
    base_model = InceptionV3(input_shape=(inputY, inputX, channel), include_top=False, weights='imagenet')

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
### DATASET GENERAL FUNCTION ###

In [ ]:
### FIT FUNKCIA ###

In [ ]:

checkpoint_path = ""
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
history = model.fit(
    trainGenerator,
    steps_per_epoch=len(trainGenerator),
    epochs=80,
    validation_data=validationGenerator,
    validation_steps=len(validationGenerator),
    callbacks=[checkpoint]
)

In [ ]:
### FIT FUNKCIA ###

In [ ]:
### GRAF ###

In [ ]:

plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
model.save("/kaggle/working/model2.h2")

In [ ]:
### GRAF ###

In [ ]:
### GENERAL TEST ###

In [ ]:
model = load_model("",compile = False)

In [ ]:
model.compile(optimizer='adam' , loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
testDatagen = ImageDataGenerator(rescale=1./255)
test_generator = testDatagen.flow_from_directory(
    '',
        target_size = (487,400),
        batch_size = 5,
        class_mode = 'binary'
    )

model.evaluate(test_generator)

In [ ]:
### GENERAL TEST ###